In [11]:
# Hidden cell for imports
from dataclasses import dataclass
import pychor
import galois
import numpy as np
from typing import List, Dict
import functools
import operator

@pychor.local_function
def shamir_share(v, t, n):
    # Step 1: pick t-1 random coefficients
    coefficients = [GF.Random() for _ in range(t-1)]

    # Step 2: construct the polynomial
    coefficients.append(GF(v))
    poly = galois.Poly(GF(coefficients))

    # Step 3: compute the shares
    shares = [ShamirShare(GF(x), poly(GF(x))) for x in range(1, n+1)]
    return shares

@pychor.local_function
def shamir_reconstruct(shares):
    # Step 1: recover the polynomial
    xs = GF([s.x for s in shares])
    ys = GF([s.y for s in shares])
    poly = galois.lagrange_poly(xs, ys)

    # Step 2: evaluate f(0) to recover the secret
    secret = poly(0)
    return secret

@dataclass
class ShamirShare:
    x: galois.GF
    y: galois.GF

    def __add__(self, other):
        assert self.x == other.x
        return ShamirShare(self.x, self.y + other.y)

    def __mul__(self, other):
        assert self.x == other.x
        return ShamirShare(self.x, self.y * other.y)

# The BGW Protocol

The Ben-Or, Goldwasser, and Wigderson protocol {cite}`ben1988completeness` is another classic MPC protocol, this time employing Shamir secret sharing and arithmetic circuits. The BGW protocol performs multiplication using the limited multiplicative homomorphism of Shamir shares. The BGW protocol naturally supports more than 2 parties, and actually *requires* at least 3 parties for the standard variant.

````{admonition} Further reading: BGW Protocol
:class: seealso

For more information on the BGW protocol, see **Section 3.3 of [Pragmatic MPC](https://securecomputation.org/)**.
````

The basic ideas behind the BGW protocol are:
- All values are field elements in $GF(p)$ for some (usually large) prime $p$
- Parties hold Shamir secret shares of values, and shares are built as described in Chapter 9
- Parties perform addition locally, via the additive homomorphism of the shares
- Parties perform multiplication using the limited multiplicative homomorphism of the shares, with a *degree reduction* step to avoid the threshold $t$ growing too large
- We start with the threshold $t = \lfloor n/2 \rfloor$ where $n$ is the number of parties, so that we can use the limited multiplicative homomorphism of the shares
- This means the protocol requires an *honest majority*: if more than half of the parties are corrupted, they can decrypt any value they like with the shares they hold

We first present the protocols for input, reveal, and multiplication, then build a BGW-based `SecInt` class that implements the BGW protocol. As a building block, we'll use the `ShamirShare` class that we defined in Chapter 9. We'll also use functions from Chapter 9 for secret sharing and reconstruction with Shamir shares.

## BGW: Input

The input protocol is simple: we secret share the input value using the `shamir_share` function from Chapter 9, and then send each share to its owner. We return a dictionary mapping each owner to their share.

In [2]:
def protocol_bgw_input(v, owner):
    # Step 1: Create Shamir shares
    shares = shamir_share(v, t, n).unlist(n)

    # Step 2: Send each share to its owner
    shares_dict = {}
    for share, p in zip(shares, parties):
        share.send(owner, p)
        shares_dict[p] = share
    return shares_dict

In [3]:
GF = galois.GF(2**31-1)
parties = [pychor.Party(f'p{i}') for i in range(1, 6)]
t = 3
n = len(parties)

with pychor.LocalBackend():
    p1 = parties[0]
    shares = protocol_bgw_input(p1.constant(5), p1)

    for p, v in shares.items():
        print(p, v)

p1 ShamirShare(x=GF(1, order=2147483647), y=GF(1477204273, order=2147483647))@{p1}
p2 ShamirShare(x=GF(2, order=2147483647), y=GF(1017476234, order=2147483647))@{p1, p2}
p3 ShamirShare(x=GF(3, order=2147483647), y=GF(768299535, order=2147483647))@{p3, p1}
p4 ShamirShare(x=GF(4, order=2147483647), y=GF(729674176, order=2147483647))@{p4, p1}
p5 ShamirShare(x=GF(5, order=2147483647), y=GF(901600157, order=2147483647))@{p5, p1}


## BGW: Reveal

The reveal protocol is similarly simple: each party broadcasts their share to the other parties, and then we call `shamir_reconstruct` on the broadcasted shares.

In [4]:
def protocol_bgw_reveal(shares):
    # Step 1: Broadcast shares to all parties
    for p1, share in shares.items():
        for p2 in parties:
            share.send(p1, p2)

    # Step 2: Reconstruct the secret
    reconstructed = shamir_reconstruct(list(shares.values()))
    return reconstructed

In [5]:
with pychor.LocalBackend():
    p1 = parties[0]
    v = p1.constant(5)
    shares = protocol_bgw_input(v, p1)
    revealed = protocol_bgw_reveal(shares)
    print('Revealed value:', revealed)

Revealed value: 5@{p3, p1, p2, p5, p4}


## BGW: Multiplication

To multiply two secret-shared values $x$ and $y$ where $t \leq \lfloor n/2 \rfloor$ and $n$ is the number of parties, each party can multiply their shares locally to get one share of the product. Unfortunately, the degree of the resulting polynomial could be as high as $n$, so we won't be able to do more than one multiplication this way.

To solve this problem, we'll use a simplified version of BGW multiplication called GRR multiplication. This approach was proposed by Gennaro, Rabin, and Rabin {cite}`gennaro1998simplified` and is commonly implemented instead of the original. GRR multiplication works by having each party multiply shares locally to obtain one share with a high threshold, then performing a *degree reduction* step to bring the threshold back down to $\lfloor n/2 \rfloor$.

To start, we assume that each party holds one share of $x$ and one share of $y$ with degree $t \leq \lfloor n/2 \rfloor$, and each party multiplies shares locally (as in Chapter 9) to obtain on share of $z = xy$ with degree $n \leq n$.

To do the degree reduction, we use the [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix#Applications), a special matrix that enables *polynomial interpolcation via matrix multiplication*. In particular, for a polynomial $f(x) = a_0 + a_1 x + ... a_n x^n$, there is a Vandermonde matrix $V$ such that the following holds:

$$
V \cdot
\begin{bmatrix}
a_0 \\
a_1 \\
...\\
a_n
\end{bmatrix}
=
\begin{bmatrix}
f(1) \\
f(2) \\
... \\
f(n)
\end{bmatrix}
$$

Notice that in our setting, the first coefficient $a_0$ *is the secret* $z$, and the right-hand side of the equation is exactly equal to the secret shares held by the parties after multipling their shares locally. To recover the secret $a_0$, if we had all of the shares $f(1) ... f(n)$, we could invert $V$ and do some algebra:

$$
\begin{bmatrix}
a_0 \\
a_1 \\
...\\
a_n
\end{bmatrix}
=
V^{-1} \cdot
\begin{bmatrix}
f(1) \\
f(2) \\
... \\
f(n)
\end{bmatrix}
$$

Since we only care about $a_0$ (and not about $a_1 \dots a_n$), we can grab just the first row of $V^{-1}$, and the dot product of that row with the shares will equal the secret! We denote the first row of $V^{-1}$ as $\lambda_1, \dots, \lambda_n$, then calculate:

$$
z = a_0 = \lambda_1 f(1) + \lambda_2 f(2) + \dots + \lambda_n f(n)
$$

At first, this doesn't seem useful, because each party only knows one share (party $i$ knows $f(i)$), and we don't want anyone to learn $z$ - we want each party to hold one share of it!

The central trick of GRR multiplication solves this problem: each party *secret-shares their share*, then computes *one share of the interpolation result* using their shares of $f(1) \dots f(n)$. This approach effectively performs *encrypted interpolation*, resulting in a *brand-new polynomial* with threshold $t$ that has new random coefficients but has $y$-intercept of $z$ (the secret we want).


````{admonition} Protocol: GRR Multiplication
:class: note

Each party $P_i$ holds one share $x_i$ of $x$ and $y_i$ of $y$. The parties $P_1, \dots, P_n$ follow the following steps:
1. Party $P_i$ computes $z_i = x_i \cdot y_i$, one share of $z$ with threshold $n$, using local multiplication of shares.
2. Party $P_i$ secret shares $z_i$ (which represents $f(i)$ from above) with threshold $t$ (usually $t = \lfloor n/2 \rfloor$), distributing one share $z_{i,j}$ to each party $j$. Party $P_j$ now holds $z_{1, j} \dots z_{n, j}$, one share of each of $z_1, \dots, z_n$.
3. Party $P_j$ locally computes $\sum_{i=1}^n \lambda_i \cdot z_{i, j}$, one share of $z$ with threshold $t$.

At the end of the protocol, each party holds one Shamir share of $z = xy$ with threshold $t$.
````

The implementation requires the ability to multiply a Shamir share by a constant (the $\lambda_i$ values), and a function to sum a list of shares.

In [8]:
@pychor.local_function
def get_y_coord(share):
    return share.y

@pychor.local_function
def mul_constant(share, c):
    return ShamirShare(share.x, share.y * c)

def sum_shares(shares):
    return functools.reduce(operator.add, shares)

def protocol_grr_mult(x_shares, y_shares):
    parties = list(x_shares.keys())
    assert list(y_shares.keys()) == parties

    # Step 1: multiply shares locally
    z_is = {pi: x_shares[pi] * y_shares[pi] for pi in parties}

    # Step 2: re-share the high-degree shares
    # z_{i,j} = zijs[pi][pj]
    zijs = {pi: protocol_bgw_input(get_y_coord(z_is[pi]), pi) for pi in parties}

    # Step 3: perform degree reduction
    Vinv = np.linalg.inv(GF(np.vander(range(1, len(parties)+1), increasing=True)))
    lambda_is = Vinv[0]

    def step3(pj):
        terms = [mul_constant(zijs[pi][pj], pj.constant(lambda_is[i])) for i, pi in enumerate(parties)]
        return sum_shares(terms)

    z_shares = {pj: step3(pj) for pj in parties}

    return z_shares

In [9]:
with pychor.LocalBackend():
    p1 = parties[0]
    p2 = parties[1]
    x = protocol_bgw_input(p1.constant(5), p1)
    y = protocol_bgw_input(p2.constant(3), p2)
    z = protocol_grr_mult(x, y)
    print(protocol_bgw_reveal(z))

15@{p3, p1, p2, p5, p4}


## BGW-Based `SecInt`

We can use the protocols defined above to define a `SecInt` class for $n$-party secure integer computations with an honest majority. We implement addition by local addition of shares, and input, multiplication, and reveal using the protocols defined earlier.

In [16]:
@dataclass
class SecInt:
    # The `shares` field maps each party to the share they hold
    shares: Dict[pychor.Party, galois.GF]

    @classmethod
    def input(cls, val):
        assert len(val.parties) == 1
        return SecInt(protocol_bgw_input(val, list(val.parties)[0]))

    def __add__(x, y):
        parties = x.shares.keys()
        assert y.shares.keys() == parties
        return SecInt({p: x.shares[p] + y.shares[p] for p in parties})

    def __mul__(x, y):
        return SecInt(protocol_grr_mult(x.shares, y.shares))

    def reveal(self):
        return protocol_bgw_reveal(self.shares)

In [17]:
with pychor.LocalBackend():
    x = SecInt.input(parties[0].constant(5))
    y = SecInt.input(parties[1].constant(3))

    print('(x+y)*x*y =', ((x+y)*x*y).reveal())

(x+y)*x*y = 120@{p3, p1, p2, p5, p4}


The BGW protocol is efficient - it natively supports arbitrary fields, performs addition without communication, and requires each party to send $n$ field elements for each multiplication (during the secret sharing of step 2 in GRR multiplication). This matches the efficiency of arithmetic multiplication-triple-based MPC as described in Chapter 4 - but *without the need for multiplication triples*.

However, the BGW protocol *requires* an honest majority assumption, which is significantly weaker than the $n-1$ assumption for protocols based on additive sharing. This also means that the protocol requires at least 3 parties: in the 2-party case, we would need to set $t = \lfloor 2/2 \rfloor = 1$, and the corrupted party would be able to reconstruct all of the secrets.

When these limitations are acceptable, however, protocols like BGW that leverage Shamir shares can be an efficient choice.